In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['nous sommes en train de perdre .', 'we re losing .']


In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    # input shape: (B, 1)
    # hidden shape: (B, hidden_size)
    # output shape: (B, hidden_size)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)

    # input shape: (B, hidden size)
    # output shape: (B, hidden size)
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output[0])
        #output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
# number of input and output vocabulary
input_size = input_lang.n_words
output_size = output_lang.n_words
hidden_size = 10
# encoder decoder


In [26]:
# dataset for translation
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class dataset(Dataset):
    def __init__(self):
        super(dataset, self).__init__()
        self.data = pairs
    
    def __len__(self):
        return len(pairs)
    
    def __getitem__(self, idx):
        input, output = pairs[idx]
        input_token = torch.tensor([input_lang.word2index[word] for word in input.split()] 
                                   + [EOS_token], device=device)
        output_token = torch.tensor([SOS_token] 
                                    + [output_lang.word2index[word] for word in output.split()] 
                                    + [EOS_token], device=device)
        return input_token, output_token
    
data = dataset()
#dataloader = DataLoader(data,batch_size=2)

In [27]:
data[1]

(tensor([6, 7, 8, 5, 1], device='cuda:0'),
 tensor([0, 2, 3, 5, 4, 1], device='cuda:0'))

In [28]:
class seq2seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(seq2seq, self).__init__()
        self.encoder = EncoderRNN(input_size, hidden_size)
        self.decoder = DecoderRNN(hidden_size,output_size)
    def train(self, dataset, epochs=1):
        criterion = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            for token_fra, token_eng in dataset:
                print(token_fra, token_eng)
                # start running encoder
                encoder_hidden = self.encoder.initHidden()
                print(encoder_hidden)
                for token in token_fra:
                    print(token)
                    encoder_output, encoder_hidden = self.encoder.forward(torch.tensor(token),
                                                                         encoder_hidden)
                    print(encoder_output, encoder_hidden)
                decoder_hidden = encoder_output
                for index, token in enumerate(token_eng[0:len(token_eng)-1]):
                    decoder_output, decoder_hidden = self.decoder.forward(torch.tensor(token),
                                                                         decoder_hidden)
                    loss = criterion(decoder_output, torch.tensor(token_eng[index+1]))

In [29]:
model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words)

In [30]:
model.train(data)

tensor([2, 3, 4, 5, 1], device='cuda:0') tensor([0, 2, 3, 4, 1], device='cuda:0')
tensor([[[0., 0., 0., 0., 0.]]], device='cuda:0')
tensor(2, device='cuda:0')


C:\Users\hongm\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


RuntimeError: Expected object of backend CPU but got backend CUDA for argument #3 'index'

In [19]:
dataset

__main__.dataset